# Анализ поведения пользователей

In [26]:
users[users['MainScreenAppear'].isna()].head()

event_name,CartScreenAppear,MainScreenAppear,OffersScreenAppear,PaymentScreenSuccessful,Tutorial
id,,,,,
74158328448226259,2019-08-01 09:32:03,NaT,2019-08-01 09:32:07,2019-08-01 09:32:02,NaT
111394506613435756,2019-08-02 15:17:27,NaT,2019-08-02 15:17:28,2019-08-02 15:17:25,NaT
214966247576341063,2019-08-01 07:04:01,NaT,2019-08-01 07:02:56,2019-08-01 07:04:00,NaT
261817378841141406,2019-08-02 10:36:04,NaT,2019-08-02 10:36:09,2019-08-02 10:36:03,NaT
332529825412858125,2019-08-01 07:13:44,NaT,2019-08-01 07:13:37,2019-08-01 16:06:20,NaT


Как мы видим общее количество пользователей(7534) не совпадает с количеством пользователей на первом шаге(7419). Исходя из данных видно что оплата предшествует просмотру страницы корзины и каталога, а время просмотра главной страницы вовсе отсутствует.

In [27]:
bug_orders = users[users['MainScreenAppear'].isna()]
len(bug_orders)

115

In [28]:
result = pd.DataFrame()

for i in bug_orders.iterrows():
    rows = data[(data['event_name'] == 'CartScreenAppear') & (data['id'] == i[0]) & (data['event_datetime'] == i[1][0])]
    result = result.append(rows)
    rows = data[(data['event_name'] == 'OffersScreenAppear') & (data['id'] == i[0]) & (data['event_datetime'] == i[1][2])]
    result =result.append(rows)
    rows = data[(data['event_name'] == 'PaymentScreenSuccessful') & (data['id'] == i[0]) & (data['event_datetime'] == i[1][3])]
    result = result.append(rows)
    rows = data[(data['event_name'] == 'Tutorial') & (data['id'] == i[0]) & (data['event_datetime'] == i[1][4])]
    result = result.append(rows)

result.groupby('event_name').agg({'id': 'count'})


,id
event_name,
CartScreenAppear,99
OffersScreenAppear,111
PaymentScreenSuccessful,98
Tutorial,4


In [29]:
bug_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 74158328448226259 to 9217594193087726423
Data columns (total 5 columns):
CartScreenAppear           99 non-null datetime64[ns]
MainScreenAppear           0 non-null datetime64[ns]
OffersScreenAppear         111 non-null datetime64[ns]
PaymentScreenSuccessful    98 non-null datetime64[ns]
Tutorial                   4 non-null datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 5.4 KB


In [30]:
data_clear = data.drop(result.index)

перезапишим таблицу users

In [31]:
users = data_clear.pivot_table(index='id',
                        columns='event_name',
                        values='event_datetime',
                        aggfunc='min')

Расчитаем воронки для всех пользователей и для каждой группы отдельно.

In [32]:
step_1 = ~users['MainScreenAppear'].isna()
step_2 = step_1 & (users['OffersScreenAppear'] > users['MainScreenAppear'])
step_3 = step_2 & (users['CartScreenAppear'] > users['OffersScreenAppear'])
step_4 = step_3 & (users['PaymentScreenSuccessful'] > users['CartScreenAppear'])

funnel = pd.DataFrame(columns=['users'],
                    index=['MainScreenAppear',
                        'OffersScreenAppear',
                        'CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel['MainScreenAppear':] = users[step_1].shape[0] 
funnel['OffersScreenAppear':] =  users[step_2].shape[0]
funnel['CartScreenAppear':] = users[step_3].shape[0]
funnel['PaymentScreenSuccessful':] = users[step_4].shape[0]

In [33]:
users_246 = data_clear.query('exp_id == 246').pivot_table(index='id',
                        columns='event_name',
                        values='event_datetime',
                        aggfunc='min')

step_1_246 = ~users_246['MainScreenAppear'].isna()
step_2_246 = step_1_246 & (users_246['OffersScreenAppear'] > users_246['MainScreenAppear'])
step_3_246 = step_2_246 & (users_246['CartScreenAppear'] > users_246['OffersScreenAppear'])
step_4_246 = step_3_246 & (users_246['PaymentScreenSuccessful'] > users_246['CartScreenAppear'])

funnel_246 = pd.DataFrame(columns=['users'],
                    index=['MainScreenAppear',
                        'OffersScreenAppear',
                        'CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel_246['MainScreenAppear':] = users_246[step_1_246].shape[0] 
funnel_246['OffersScreenAppear':] =  users_246[step_2_246].shape[0]
funnel_246['CartScreenAppear':] = users_246[step_3_246].shape[0]
funnel_246['PaymentScreenSuccessful':] = users_246[step_4_246].shape[0]

funnel_246

,users
MainScreenAppear,2450
OffersScreenAppear,1411
CartScreenAppear,584
PaymentScreenSuccessful,145


In [34]:
users_247 = data_clear.query('exp_id == 247').pivot_table(index='id',
                        columns='event_name',
                        values='event_datetime',
                        aggfunc='min')

step_1_247 = ~users_247['MainScreenAppear'].isna()
step_2_247 = step_1_247 & (users_247['OffersScreenAppear'] > users_247['MainScreenAppear'])
step_3_247 = step_2_247 & (users_247['CartScreenAppear'] > users_247['OffersScreenAppear'])
step_4_247 = step_3_247 & (users_247['PaymentScreenSuccessful'] > users_247['CartScreenAppear'])

funnel_247 = pd.DataFrame(columns=['users'],
                    index=['MainScreenAppear',
                        'OffersScreenAppear',
                        'CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel_247['MainScreenAppear':] = users_247[step_1_247].shape[0] 
funnel_247['OffersScreenAppear':] =  users_247[step_2_247].shape[0]
funnel_247['CartScreenAppear':] = users_247[step_3_247].shape[0]
funnel_247['PaymentScreenSuccessful':] = users_247[step_4_247].shape[0]

funnel_247

,users
MainScreenAppear,2476
OffersScreenAppear,1379
CartScreenAppear,600
PaymentScreenSuccessful,144


In [35]:
users_248 = data_clear.query('exp_id == 248').pivot_table(index='id',
                        columns='event_name',
                        values='event_datetime',
                        aggfunc='min')

step_1_248 = ~users_248['MainScreenAppear'].isna()
step_2_248 = step_1_248 & (users_248['OffersScreenAppear'] > users_248['MainScreenAppear'])
step_3_248 = step_2_248 & (users_248['CartScreenAppear'] > users_248['OffersScreenAppear'])
step_4_248 = step_3_248 & (users_248['PaymentScreenSuccessful'] > users_248['CartScreenAppear'])

funnel_248 = pd.DataFrame(columns=['users'],
                    index=['MainScreenAppear',
                        'OffersScreenAppear',
                        'CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel_248['MainScreenAppear':] = users_248[step_1_248].shape[0] 
funnel_248['OffersScreenAppear':] =  users_248[step_2_248].shape[0]
funnel_248['CartScreenAppear':] = users_248[step_3_248].shape[0]
funnel_248['PaymentScreenSuccessful':] = users_248[step_4_248].shape[0]

funnel_248

,users
MainScreenAppear,2493
OffersScreenAppear,1411
CartScreenAppear,583
PaymentScreenSuccessful,165


In [36]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    y = funnel.index,
    x = funnel['users'],
    textinfo = 'value+percent initial'
))

fig.show()

In [37]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = '246',
    y = funnel_246.index,
    x = funnel_246['users'],
    textinfo = 'value+percent initial'
))

fig.add_trace(go.Funnel(
    name = '247',
    y = funnel_247.index,
    x = funnel_247['users'],
    textinfo = 'value+percent initial'
))

fig.add_trace(go.Funnel(
    name = '248',
    y = funnel_248.index,
    x = funnel_248['users'],
    textinfo = 'value+percent initial'
))

fig.show()

Больше всего пользователей теряется после главного экрана, 6,1% от всех пользователей доходит от главного экрана, по группам 246, 247, 248 от главного экрана до покупки доходит 5,9% 5,8% и 6,6% соответственно.

Построим воронки для вожможных вариаций пользовательского пути.

In [38]:
step_1 = ~users['OffersScreenAppear'].isna()
step_2 = step_1 & (users['CartScreenAppear'] > users['OffersScreenAppear'])
step_3 = step_2 & (users['PaymentScreenSuccessful'] > users['CartScreenAppear'])

funnel_2 = pd.DataFrame(columns=['users'],
                    index=['OffersScreenAppear',
                        'CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel_2['OffersScreenAppear':] =  users[step_1].shape[0]
funnel_2['CartScreenAppear':] = users[step_2].shape[0]
funnel_2['PaymentScreenSuccessful':] = users[step_3].shape[0]

fig = go.Figure()

fig.add_trace(go.Funnel(
    y = funnel_2.index,
    x = funnel_2['users'],
    textinfo = 'value+percent initial'
))

fig.show()


Данная воронка отражает только пользователей прошедших путь 'OffersScreenAppear' - 'CartScreenAppear' - 'PaymentScreenSuccessful'

In [39]:
step_1 = ~users['CartScreenAppear'].isna()
step_2 = step_1 & (users['PaymentScreenSuccessful'] > users['CartScreenAppear'])

funnel_3 = pd.DataFrame(columns=['users'],
                    index=['CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel_3['CartScreenAppear':] = users[step_1].shape[0]
funnel_3['PaymentScreenSuccessful':] = users[step_2].shape[0]

fig = go.Figure()

fig.add_trace(go.Funnel(
    y = funnel_3.index,
    x = funnel_3['users'],
    textinfo = 'value+percent initial'
))

fig.show()


Данная воронка отражает только пользователей прошедших путь 'CartScreenAppear' - 'PaymentScreenSuccessful'

In [40]:
step_1 = ~users['MainScreenAppear'].isna()
step_2 = step_1 & (users['CartScreenAppear'] > users['MainScreenAppear'])
step_3 = step_2 & (users['PaymentScreenSuccessful'] > users['CartScreenAppear'])

funnel_4 = pd.DataFrame(columns=['users'],
                    index=['MainScreenAppear',
                        'CartScreenAppear',
                        'PaymentScreenSuccessful'])

funnel_4['MainScreenAppear':] = users[step_1].shape[0]
funnel_4['CartScreenAppear':] = users[step_2].shape[0]
funnel_4['PaymentScreenSuccessful':] = users[step_3].shape[0]

fig = go.Figure()

fig.add_trace(go.Funnel(
    y = funnel_4.index,
    x = funnel_4['users'],
    textinfo = 'value+percent initial'
))

fig.show()

Данная воронка отражает только пользователей прошедших путь 'MainScreenAppear' - 'CartScreenAppear' -  'PaymentScreenSuccessful'

### Вывод

Мы видим что далеко не все пользователи проходят весь путь. Однако средняя конверсия около 6 процентов. Есть часть пользователей которые проходят путь корзина - оплата, что странно, так как это приложение нельзя зайти напрямую в корзину, скорее всего где-то есть ошибки при записи данных. Так как большая часть пользователей отваливается после главного экрана возможно стоит обратить внимание на другие метрики чтобы изучить пользовательский опыт и понять что именно не нравиться пользователям и как можно увеличить конверсию.

<a id='s4'> </a>

## Результаты эксперимента

In [41]:
print('Количество пользователей в группах: \n246: {} пользователей \n247: {} пользователей \n248: {} пользователей\nИтого: {}'.format(
    users_246.shape[0],
    users_247.shape[0],
    users_248.shape[0],
    users.shape[0]
))

Количество пользователей в группах: 
246: 2478 пользователей 
247: 2511 пользователей 
248: 2530 пользователей
Итого: 7519


In [42]:
orders_246 = (data_clear[(data_clear['exp_id'] == 246) &
    (data_clear['event_name'] == 'PaymentScreenSuccessful')]
    .groupby('id', as_index=False)
    .agg({'date': 'count'})
)

orders_247 = (data_clear[(data_clear['exp_id'] == 247) &
    (data_clear['event_name'] == 'PaymentScreenSuccessful')]
    .groupby('id', as_index=False)
    .agg({'date': 'count'})
)

orders_248 = (data_clear[(data_clear['exp_id'] == 248) &
    (data_clear['event_name'] == 'PaymentScreenSuccessful')]
    .groupby('id', as_index=False)
    .agg({'date': 'count'})
)

Проверим количество событий в группах.

In [43]:
(orders_246['date'].sum() + orders_247['date'].sum() + orders_248['date'].sum())\
    - len(data_clear[data_clear['event_name'] == 'PaymentScreenSuccessful'])

0

Количество событий совпадает.

Проверим есть ли пользователи находяшиеся в нескольких группах.

In [44]:
aab_users = data_clear.groupby('id', as_index=False).agg({'exp_id': 'nunique'})
aab_users = aab_users[aab_users['exp_id'] > 1]['id']
len(aab_users)

0

In [45]:
def z_test(first, second):

    # так как мы проводим 23 теста воспользуемся методом Шидака для расчета значения альфа
    alpha = 1 - (1 - 0.05)**(1/23)

    next = np.array(second)
    privious = np.array(first)

    p1 = next[0] / privious[0]
    p2 = next[1] / privious[1]

    p_combined = next.sum() / privious.sum()

    difference = p1 - p2

    z_value = difference / mth.sqrt(
        p_combined * (1-p_combined) * (1/privious[0] + 1/privious[1]))

    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-value:', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')


Проверим есть ли статистически значимые различия между выборками. 

Н0 - статистически значимых различий между долями нет

Н1 - есть статистически значимая разница между долями

Проведем z-тест между для двух групп на каждом шаге воронки.

In [46]:
z_test(
    [funnel_246.loc['MainScreenAppear','users'], funnel_247.loc['MainScreenAppear','users']],
    [funnel_246.loc['OffersScreenAppear','users'], funnel_247.loc['OffersScreenAppear','users']]
)

p-value: 0.1791392160319234
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [47]:
z_test(
    [funnel_246.loc['OffersScreenAppear','users'], funnel_247.loc['OffersScreenAppear','users']],
    [funnel_246.loc['CartScreenAppear','users'], funnel_247.loc['CartScreenAppear','users']]
)

p-value: 0.2571602924470018
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [48]:
z_test(
    [funnel_246.loc['CartScreenAppear','users'], funnel_247.loc['CartScreenAppear','users']],
    [funnel_246.loc['PaymentScreenSuccessful','users'], funnel_247.loc['PaymentScreenSuccessful','users']]
)

p-value: 0.7399516568824782
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Исходя из результатов тестов мы не можем отвергнуть нулевую гипотезу о том что статистически значимой разницы между выборками 246 и 247 нет.

In [49]:
data_clear.groupby('event_name', as_index=False).agg({'id': 'count'}).sort_values('id', ascending=False)

,event_name,id
1,MainScreenAppear,117328
2,OffersScreenAppear,46222
0,CartScreenAppear,42204
3,PaymentScreenSuccessful,33820
4,Tutorial,1001


Самое популярное событие MainScreenAppear. Узнаем число пользователей совршивших это событие в каждой из групп.

In [50]:
def event_parts(event_name):
    event_name = data_clear.query('event_name == @event_name')\
        .groupby('exp_id', as_index=False)\
        .agg({'id': 'nunique'})
    event_name = event_name.rename(columns={'id': 'users_part'})

    group_users = data_clear.groupby('exp_id', as_index=False)\
        .agg({'id': 'nunique'})
    group_users = group_users.rename(columns={'id': 'all_users'})

    event_name = group_users.merge(
        event_name,
        on='exp_id',
        how='left')

    event_name['ratio'] = event_name['users_part']\
        / event_name['all_users']

    return event_name



In [51]:
event_parts('MainScreenAppear')

,exp_id,all_users,users_part,ratio
0,246,2478,2450,0.988701
1,247,2511,2476,0.986061
2,248,2530,2493,0.985375


<a id='s5'> </a>

### Z-Тесты для событий

Посмотрим есть ли статистически значимая разница между долями.

Н0 - статистически значимых различий между долями нет<br>
Н1 - есть статистически значимая разница между долями


In [52]:
def test(group_1, group_2, event_users):
    z_test(
        [
            int(event_users[event_users['exp_id'] == group_1]['all_users']),
            int(event_users[event_users['exp_id'] == group_2]['all_users'])
        ],
        [
            int(event_users[event_users['exp_id'] == group_1]['users_part']),
            int(event_users[event_users['exp_id'] == group_2]['users_part'])
        ]
    )

#### MainScreenAppear

In [53]:
test(246, 247, event_parts('MainScreenAppear'))

p-value: 0.40387484283804476
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [54]:
test(246, 248, event_parts('MainScreenAppear'))

p-value: 0.29860841248209
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [55]:
test(247, 248, event_parts('MainScreenAppear'))

p-value: 0.8374217059128042
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Исходя из результатов тестов мы не можем отвергнуть нулевую гипотезу о том что статистически значимой разницы между долями 246, 247 и 248 нет.

Сделаем тоже самое для других событий

#### OffersScreenAppear

In [56]:
event_parts('OffersScreenAppear')

,exp_id,all_users,users_part,ratio
0,246,2478,1537,0.620258
1,247,2511,1517,0.604142
2,248,2530,1524,0.602372


In [57]:
test(246, 247, event_parts('OffersScreenAppear'))

p-value: 0.24277036195922763
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [58]:
test(246, 248, event_parts('OffersScreenAppear'))

p-value: 0.19420032981963087
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [59]:
test(247, 248, event_parts('OffersScreenAppear'))

p-value: 0.8977892380394279
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


#### CartScreenAppear

In [60]:
event_parts('CartScreenAppear')

,exp_id,all_users,users_part,ratio
0,246,2478,1257,0.507264
1,247,2511,1233,0.491039
2,248,2530,1224,0.483794


In [61]:
test(246, 247, event_parts('CartScreenAppear'))

p-value: 0.2518120159282209
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [62]:
test(246, 248, event_parts('CartScreenAppear'))

p-value: 0.09674381977715152
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [63]:
test(247, 248, event_parts('CartScreenAppear'))

p-value: 0.6068662291594062
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


#### PaymentScreenSuccessful

In [64]:
event_parts('PaymentScreenSuccessful')

,exp_id,all_users,users_part,ratio
0,246,2478,1191,0.480630
1,247,2511,1151,0.458383
2,248,2530,1175,0.464427


In [65]:
test(246, 247, event_parts('PaymentScreenSuccessful'))

p-value: 0.11543227441042037
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [66]:
test(246, 248, event_parts('PaymentScreenSuccessful'))

p-value: 0.25084439500508626
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [67]:
test(247, 248, event_parts('PaymentScreenSuccessful'))

p-value: 0.6669137994898544
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


#### Tutorial

In [68]:
event_parts('Tutorial')

,exp_id,all_users,users_part,ratio
0,246,2478,277,0.111784
1,247,2511,283,0.112704
2,248,2530,278,0.109881


In [69]:
test(246, 247, event_parts('Tutorial'))

p-value: 0.9179863365481813
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [70]:
test(246, 248, event_parts('Tutorial'))

p-value: 0.8302280456901241
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [71]:
test(247, 248, event_parts('Tutorial'))

p-value: 0.7500091264489663
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


<a id='s6'> </a>

### Объединенная группа

Проведем тесты для объединенной контрольной группы.

In [72]:
def grouped_parts(event_name):
    data_def = data_clear.copy()
    data_def.loc[data_def['exp_id'] == 246, 'exp_id'] = 'grouped'
    data_def.loc[data_def['exp_id'] == 247, 'exp_id'] = 'grouped'
    event_name = data_def.query('event_name == @event_name')\
        .groupby('exp_id', as_index=False)\
        .agg({'id': 'nunique'})
    event_name = event_name.rename(columns={'id': 'users_part'})

    group_users = data_def.groupby('exp_id', as_index=False)\
        .agg({'id': 'nunique'})
    group_users = group_users.rename(columns={'id': 'all_users'})

    event_name = group_users.merge(
        event_name,
        on='exp_id',
        how='left')

    event_name['ratio'] = event_name['users_part']\
        / event_name['all_users']

    return event_name

In [73]:
grouped_parts('MainScreenAppear')

,exp_id,all_users,users_part,ratio
0,248,2530,2493,0.985375
1,grouped,4989,4926,0.987372


In [74]:
test(248, 'grouped', grouped_parts('MainScreenAppear'))

p-value: 0.47513123512740707
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [75]:
test(248, 'grouped', grouped_parts('OffersScreenAppear'))

p-value: 0.4118149791581025
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [76]:
test(248, 'grouped', grouped_parts('CartScreenAppear'))

p-value: 0.20979645472177788
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [77]:
test(248, 'grouped', grouped_parts('PaymentScreenSuccessful'))

p-value: 0.6810305691739882
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [78]:
test(248, 'grouped', grouped_parts('Tutorial'))

p-value: 0.7580929728849777
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


<a id='s7'> </a>

# Вывод 

Исходя из данных мы узнали что пользователи разняться в своем пути до успешной оплаты. Возможно некоторые по рекламным ресурсам напрямую в каталог. При этом у пользователей прошедших весь путь конверсия около 6% независимо от группы эксперемента.<br>
<br>
Так как большинство пользователей отваливается после гланого экрана следует обратить внимание на другие метрики чтобы изучить пользовательский опыт, понять как можно его улучшить и увеличить конверсию.<br> 
<br>
Мы провели тесты для каждого события и группы и исходя из результатов тестов мы не можем отвергнуть нулевую гипотезу о том что статистически значимой разницы между долями 246, 247 и 248 нет.<br>
<br>
Исходя из этого мы можем предположить что изменение шрифта не влияет на конверсию.